In [13]:
import numpy as np
import h5py
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/My Drive/Colab_Notebooks/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
hf = h5py.File(path + 'MNIST_synthetic.h5', 'r')
print(hf.keys())

X_train = hf['train_dataset']
print(X_train.shape)

y_train = hf['train_labels']
print(y_train.shape)

X_test = hf['test_dataset']
print(X_test.shape)

<KeysViewHDF5 ['test_dataset', 'train_dataset', 'train_labels']>
(56000, 64, 64, 1)
(56000, 5)
(14000, 64, 64, 1)
